# Summarize WSF

The World Settlement Footprint was officially launched in November 2021, [see the announcement here for more details](https://www.esa.int/Applications/Observing_the_Earth/Mapping_our_human_footprint_from_space).

GOST have downloaded the evolution dataset, generated a VRT combining the ~5000 tiles, and uploaded to our private AWS bucket (wbgdecinternal-ntl). That dataset will be used to summarize change in the built up area in a variety of urban areas

In [2]:
import sys, os, importlib
import rasterio

import pandas as pd
import geopandas as gpd

try:
    import GOSTRocks.rasterMisc as rMisc    
    from GOSTRocks.misc import tPrint
except:
    print("Cannot find GOSTRocks")


In [3]:
wsf_evolution = 's3://wbgdecinternal-ntl/GIS_Data/GLOBAL/WSFEvolution/WSF_Evolution.vrt'
urban_extents = '/home/public/Data/GLOBAL/URBAN/GHS/GHS_STAT_UCDB2015MT_GLOBE_R2019A/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg'
out_folder = "/home/wb411133/data/Projects/SD_FLAGSHIP/Data/GHS_URB_RES"
WSF_summary_file = os.path.join(out_folder, "WSF_Evolution.csv")

#Columns in urban database to keep in output - includes ID column and names
inD_cols = ['ID_HDC_G0','CTR_MN_NM','CTR_MN_ISO','UC_NM_MN', 'UC_NM_LST'] 
unq_vals = [0] + list(range(1985, 2016))

In [4]:
inR = rasterio.open(wsf_evolution)
inD = gpd.read_file(urban_extents)

In [5]:
if not os.path.exists(WSF_summary_file):
    res = rMisc.zonalStats(inD, inR, rastType='C', unqVals=unq_vals)

    named = pd.DataFrame(res, columns=[f'c_{x}' for x in unq_vals])
    for col in inD_cols:
        named[col] = inD[col]

    named.to_csv(WSF_summary_file)

# Map some results

In [6]:
from IPython.display import display, Markdown, HTML, FileLink, FileLinks
import folium
from matplotlib import cm

In [7]:
# Select a single city in the dataset
sel_city = inD.loc[inD['UC_NM_MN'] == "Washington D.C."]
sel_city

,ID_HDC_G0,QA2_1V,AREA,BBX_LATMN,BBX_LONMN,BBX_LATMX,BBX_LONMX,GCPNT_LAT,GCPNT_LON,CTR_MN_NM,...,EX_SS_P00,EX_SS_P15,EX_EQ19PGA,EX_EQ19MMI,EX_EQ19_Q,EX_HW_IDX,SDG_LUE9015,SDG_A2G14,SDG_OS15MX,geometry
854,855.0,1.0,1550.0,38.685968,-77.56703,39.219741,-76.802698,38.92284,-77.141963,United States,...,968446.778435,1.066668e+06,0.020453,3.0,available,5.99002,1.2541,0.661224,39.8,"MULTIPOLYGON (((-77.27252 39.21974, -77.26083 ..."


In [8]:
#Extract WSF for the desired area
geometry = sel_city.unary_union
ul = inR.index(*geometry.bounds[0:2])
lr = inR.index(*geometry.bounds[2:4])
window = ((float(lr[0]), float(ul[0]+1)), (float(ul[1]), float(lr[1]+1)))

inR_data = inR.read(1, window=window, boundless=True, fill_value=0)


In [11]:
m = folium.Map([geometry.centroid.y, geometry.centroid.x], zoom_start=10, tiles='stamentoner')
b = geometry.bounds

folium.raster_layers.ImageOverlay(
    image = inR_data,
    bounds = [[b[1],b[0]],[b[3],b[2]]],
    colormap=cm.magma # Colour mapping is still terrible; help please
).add_to(m)

simple_geo = gpd.GeoSeries(geometry).to_json()
geo_folium = folium.GeoJson(data=simple_geo, 
                           style_function=lambda x: {'fillColor': '#00000000'})
geo_folium.add_to(m)

m